In [1]:
import networkx as nx
import sys
fn='c432'

**The required 'networkx' library is imported, which can be used to create the required graphs.**

In [2]:
with open(fn+'.net','r') as f: #CHANGE FILENAME HERE
    circ= f.readlines()
circ=[i.rstrip('\n') for i in circ]
#circ

**The netlist file is read in the form of multiple lists using ```readlines()```method.**

In [3]:
l=[]
d={}
qu=[]
set1=set()
set2=set()
for i in circ:
    i=i.split()
    l.append((i[2],i[-1])) #appending the first input and output of a gate
    if i[1][-1]=='2':
        l.append((i[3],i[-1])) #appending the second input and output of a gate
        set1.add(i[3])  #includes the second input to a gate if exists, and the first input
    set1.add(i[2])  #contains the set of all nets
    set2.add(i[-1]) #contains the set of outputs from all gates
    
for i in circ:
    i=i.split()
    d[i[-1]]=i[1] #mapping the output nets to their respective gates
for i in set1 - set2: #set1 - set2 contains the set of primary inputs
    d[i]="PI"  #mapping primary inputs to 'PI'
    qu.append(i) #queue containing all the primary inputs
#print(l,d)

* **A list containing all the edges of the graph, and the sets of all nets and output nets are constructed.**
* **A dictionary mapping the nets to their respective gates is also created, along with a queue containing all the primary inputs.**

In [4]:
try:
    g = nx.DiGraph()
    g.add_edges_from(l) #graph generated using the list of edges 
    nx.set_node_attributes(g, d,name="gateType")
    #print(g.nodes(data=True))
    nl = (list(nx.topological_sort(g))) #list containing the nets in the topological order 
    #print('Nodes in topological order',nl)
except:
    print("Error! The graph contains a cycle!")
    sys.exit(0)

* **Directed Acyclic Graph (DAG) is also created using the networkx library using the list of edges and the mapping of nets to the respective gates.**
* **The networkx library generates the list of nets in topological order using ```nx.topological_sort()```.**

In [5]:
with open(fn+'.inputs','r') as f: # CHANGE FILENAME HERE
    values= f.readlines()
values=[i.rstrip('\n') for i in values]
#values

**The file containing the values of the primary inputs (PI) at different instants of time is also read in the above mentioned way.**

In [6]:
dv={}
i=values[0].split() #list of primary inputs (can be used directly instead of using sets)
for dum in i:
    dv[dum]=[]
for j in range(1,len(values)):
    values[j]=values[j].split()
for j in range(len(i)):
    for k in range(1,len(values)):
        dv[i[j]].append(int(values[k][j])) #list of the values of primary inputs at different instants
#print(dv)

**The list of values of the primary inputs at different instants is analysed and only the changing inputs are appended into a list.**

# Topologically ordered evaluation


In [7]:
#print(sorted(nl))
def funct():    
    l=[]
    for j in range(len(values)-1):
        ans={}
        for i in nl:
            dum=list(g.predecessors(i)) # extracting the inputs for the net value to be evaluated
            if d[i]=='PI':
                ans[i]=int(dv[i][j])
            elif d[i]=='nand2':
                ans[i]=int(not(ans[dum[0]]&ans[dum[1]]))
            elif d[i]=='and2':
                ans[i]=int(ans[dum[0]]&ans[dum[1]])
            elif d[i]=='or2':
                ans[i]=int(ans[dum[0]]|ans[dum[1]])
            elif d[i]=='nor2':
                ans[i]=int(not(ans[dum[0]]|ans[dum[1]]))
            elif d[i]=='xor2':
                ans[i]=int(ans[dum[0]]^ans[dum[1]])
            elif d[i]=='xnor2':
                ans[i]=int(not(ans[dum[0]]^ans[dum[1]]))
            elif d[i]=='buf':
                ans[i]=int(ans[dum[0]])
            elif d[i]=='inv':
                ans[i]= int(not(ans[dum[0]]))
        #print(list(dict(sorted(ans.items())).values()))
        l.append(list(dict(sorted(ans.items())).values())) # appending the outputs at different instants


    #print(l)
    f=open('apl4_top','w')
    f.writelines(' '.join(sorted(nl))) #the elements of list nl are joined using a string ' '
    for i in range(len(l)):
        l[i]=[str(x) for x in l[i]]
        f.write('\n')
        f.writelines(' '.join(l[i]))
    f.close()  
%timeit funct()

57.4 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


* **The list nl, containing the nets in topological order is iterated through and the values of all nets are evaluated and stored in a list.**
* **The corresponding gate names are checked for, using ```if-else```; but can also be done using ```eval()``` function in the following way:**
```python
def nand2(x,y):
    return int(not(x&y))
def and2(x,y):
    return int(x&y)
def or2(x,y):
    return int(x|y)
def nor2(x,y):
    return int(not(x|y))
def xor2(x,y):
    return int(x^y)
def xnor2(x,y):
    return int(not(x^y))
def buf(x,y):
    return int(x)
def inv(x,y):
    return  int(not(x))
for i in nl:
    dum=list(g.predecessors(i))
    if d[i]=='PI':
        ans[i]=int(dv[i][j])
    else:
        z=ans[dum[0]]
        try:
            y=ans[dum[1]]
        except:
            y=0
        ans[i]=eval(d[i])(z,y)
```
* **The ```eval()``` function parses the expression argument and evaluates it as a python expression and runs python expression (code) within the program.**
* **The ```eval()``` function has to read the code, convert it to bytecode during the time of execution, each and every time in the loop, which adds to the execution time, which is why it is not used in the main code, even though it simplifies the code.**
* **A new file 'apl4_top' is then opened in write 'w' mode, and the list of nets and their corresponding values are written into it in the alphabetical order.**


# Event-driven evaluation

In [8]:
def func():
    q=qu.copy() #q=qu doesnt work as it will create a map; hence a copy is created
    ans={}
    for i in set1 | set2:
        ans[i]='x' #value of the net is given as 'x', which is equivalent to saying the net is not evaluated yet
    l=[]
    w=values[0].split()    
    for j in range(len(values)-1):
        if j>0: 
            q=[] #list or dequeue, containing all the primary inputs whose values change from one instant to the next
            for k in range(len(w)):
                if values[j+1][k]!=values[j][k]:
                    q.append(w[k])
        #print(change)            
        #print(len(q))
        while len(q)>0:
            i=q.pop(0) #removing the first element from the queue
            dum=list(g.predecessors(i))
            try: 
                if d[i]=='PI':
                    x=int(dv[i][j])    
                elif d[i]=='nand2':
                    x=int(not(ans[dum[0]]&ans[dum[1]]))
                elif d[i]=='and2':
                    x=int(ans[dum[0]]&ans[dum[1]])
                elif d[i]=='or2':
                    x=int(ans[dum[0]]|ans[dum[1]])
                elif d[i]=='nor2':
                    x=int(not(ans[dum[0]]|ans[dum[1]]))
                elif d[i]=='xor2':
                    x=int(ans[dum[0]]^ans[dum[1]])
                elif d[i]=='xnor2':
                    x=int(not(ans[dum[0]]^ans[dum[1]]))
                elif d[i]=='buf':
                    x=int(ans[dum[0]])
                elif d[i]=='inv':
                    x= int(not(ans[dum[0]]))
                if ans[i]!=x: #checking if the previous state of the net is not equal to the present state
                    ans[i]=x #changing the present state of the net
                    for dummy in list(g.successors(i)): #adding the list of successors of the net to the queue
                        q.append(dummy)  #if there is a change in its value
            except:
                #try except block used because values of some nets cant be evaluated 
                #as their inputs values might be 'x', i.e. not evaluated
                pass
            #q.pop(0)
            #print(dum)

            #print(q)
        #print(ans.keys())
        #print(ans.values())
        l.append(list(dict(sorted(ans.items())).values()))
        
        #return ans.values()
    f=open('apl4_eve','w')
    f.writelines(' '.join(sorted(nl))) #the elements of list are joined using a string ' '
    for i in range(len(l)):
        l[i]=[str(x) for x in l[i]]
        f.write('\n')
        f.writelines(' '.join(l[i]))
    f.close()  
%timeit func()
#func()

56.4 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


* **Initially, a queue containing the primary inputs is iterated through and the values of the nets are evaluated; if a change in value is observed, its successors are also added to the queue.**
* **To optimize the algorithm, the intial queue can start with only those primary inputs whose values change from one instant to the next.**
* **The values of all nets are evaluated at different instants and then written into a file named 'apl4_eve'.**

# INFERENCE -Comparing efficiencies

In [9]:
d={}
d['Filename']=['c432','c17','c8','parity']
d['Topological Sort']=['57.4 ms','0.893 ms','2.76 ms','1.04 ms']
d['Event-driven']=['56.4 ms','0.783 ms','3.26 ms','1.12 ms']
import pandas as pd
df = pd.DataFrame(d)
display(df)

Filename Topological Sort Event-driven
0     c432          57.4 ms      56.4 ms
1      c17         0.893 ms     0.783 ms
2       c8          2.76 ms      3.26 ms
3   parity          1.04 ms      1.12 ms

* **`%timeit` is used to predict the runtime of both the functions `func()` and `funct()`, which contains the core part of the topological sort and event driven algorithms; and hence can be used to infer the efficiency of both the algorithms for specific inputs.**
* **Generally, event-driven evaluation is more efficient for large networks, i.e. graphs containing large number of nets and topological sorting is more efficient for lesser number of nets.** 
* **It can be inferred that event-driven method has a clear edge over topological sort in 'c17', as the rate of toggle of the values of the primary inputs is comparatively less, i.e. the primary inputs remain at either '0' or '1' for longer duration. This makes event-driven method more efficient as it checks for the change in inputs and only adds them to a queue, whereas the topological sort goes through the entire process over and over again.**
* **The topological sort has a clear edge over event-driven method for 'c8' and 'parity' as the rate of toggling of the values of the primary inputs is high, i.e. the values keep toggling quite frequently; which leads to the entire set of primary inputs added to the queue, thereby computing all the values again.**
* **Both the methods are more or less equally efficient in 'c432'; as the large number of nets favours event-driven method and the higher rate of toggling favours topological sort.**
* **Another deciding factor could be the kind of dependence of nets on its inputs. For eg, in a file containing many 'NOT' gates, where the `inv` gate's output net is triggered only if its single input is changed, naturally has lesser probability to get evaluated again than an 'AND' gate with two input nets.** 